In [1]:
pwd

'd:\\GenAI_Projects\\End-to-End-GenAI-App-CICD-AWS-Deployment\\research'

In [2]:
import os
os.chdir("../")

In [3]:
pwd

'd:\\GenAI_Projects\\End-to-End-GenAI-App-CICD-AWS-Deployment'

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [6]:
pwd

'd:\\GenAI_Projects\\End-to-End-GenAI-App-CICD-AWS-Deployment'

In [7]:
extracted_data=load_pdf_file(data='data/')

In [8]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 1271


In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

In [11]:
#Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [12]:
embeddings = download_hugging_face_embeddings()

C:\Users\pubud\AppData\Local\Temp\ipykernel_9568\2661704553.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
d:\GenAI_Projects\End-to-End-GenAI-App-CICD-AWS-Deployment\genai\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [13]:
query_result = embeddings.embed_query("Hello")
print("Length", len(query_result))

Length 384


In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
pwd

'd:\\GenAI_Projects\\End-to-End-GenAI-App-CICD-AWS-Deployment'

In [16]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [17]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "signalragbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
) 

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': 'a91f87a0fe74052b430807c215c71279', 'Date': 'Wed, 04 Dec 2024 14:57:15 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [18]:
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [19]:
# Embed each chunk and upsert the embeddings into your Pinecone index.
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [20]:
# Load Existing index 

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [21]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [22]:
retrieved_docs = retriever.invoke("What is BERT?")

In [23]:
retrieved_docs

[Document(id='a9ac5101-79a6-473f-becf-4d2d4efaf198', metadata={'page': 7.0, 'source': 'data\\An_Explainable_Transformer-Based_Deep_Learning_Model_for_the_Prediction_of_Incident_Heart_Failure.pdf'}, page_content='ration element (“SEP”). Similar to the original BERT model,\nwe implemented an annotation ordering the sequential medical\nhistory data. Furthermore, we added layers of information that\ninvolve age and calendar year of each encounter. Thus, the total\ninput comprised of three layers of information for each and every\nencounter: the encounter itself (diagnoses and/or medications),\nage, and calendar year.\nB. Details of Perturbation Methods\nAdditionally to Guan et al. ’s method [11], we developed an'),
 Document(id='9b940518-263b-4acd-b06d-19c84f9123cd', metadata={'page': 10.0, 'source': 'data\\An_Explainable_Transformer-Based_Deep_Learning_Model_for_the_Prediction_of_Incident_Heart_Failure.pdf'}, page_content='[19] J. Devlin, M.-W. Chang, K. Lee, K. T. Google, and A. I. Lan-\

In [24]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [25]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [26]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [27]:
response = rag_chain.invoke({"input": "what is BERT?"})
print(response["answer"])



BERT is a deep bidirectional transformer model that was pre-trained for language understanding by Google in 2019. It is designed to handle sequential data, such as medical history, and can incorporate additional layers of information such as age and calendar year. BERT has been used in various applications, including visual analytics and malware detection.
